In [ ]:
import glob

import calplot
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Get all power consumption files
csv_filenames = glob.glob("data/*/stromverbrauch.csv")

# Concatinate all power consumption files, assuming that sorting their file paths in alphabetical order brings them into the correct order
csv_dfs = []
for csv_filename in sorted(csv_filenames):
    csv_df = pd.read_csv(csv_filename, delimiter=";")
    csv_dfs.append(csv_df)
combined_csv_df = pd.concat(csv_dfs, ignore_index=True)

# Date is stored either in 'Uhrzeit' or 'Uhrzeit '
combined_csv_df["Uhrzeit"] = combined_csv_df["Uhrzeit"].combine_first(
    combined_csv_df["Uhrzeit "]
)
combined_csv_df = combined_csv_df.drop(["Uhrzeit "], axis=1)

# Some values are duplicated because the are in the csv of the previous month
combined_csv_df = combined_csv_df[
    ~(combined_csv_df == combined_csv_df.shift(1)).all(axis=1)
]

# Define characteristics of the power consumption data
anomalie_limit = 10_000
unit_factor = 50
usage_unit = "kWh"
current_unit = "kW"
usage_name = "Stromverbrauch"
current_name = "Leistung"

In [ ]:
# Get all gas usage files
csv_filenames = glob.glob("data/*/gasverbrauch.csv")

# Concatinate all gas usage files, assuming that sorting their file paths in alphabetical order brings them into the correct order
csv_dfs = []
for csv_filename in sorted(csv_filenames):
    csv_df = pd.read_csv(csv_filename, delimiter=";")
    csv_dfs.append(csv_df)
combined_csv_df = pd.concat(csv_dfs, ignore_index=True)

# Some values are duplicated because the are in the csv of the previous month
combined_csv_df = combined_csv_df[combined_csv_df["Zaehlerstand"] != "0"]
combined_csv_df = combined_csv_df[
    ~(combined_csv_df == combined_csv_df.shift(1)).all(axis=1)
]

# Define characteristics of the gas usage data
anomalie_limit = 100_000
unit_factor = 1
usage_unit = "m³"
current_unit = "m³/h"
usage_name = "Gasverbrauch"
current_name = "Gasdurchfluss"

In [ ]:
# Date is stored in column 'Datum' of format '2023-04-12'
date = combined_csv_df["Datum"]

# Wall clock time is stored in column 'Uhrzeit' of format ' 1:00:00 ' and '13:00:00 '
time_of_day_raw = combined_csv_df["Uhrzeit"]
time_of_day = time_of_day_raw.str.removesuffix(" ").str.replace(" ", "0")

# Detect change daylight saving transitions
time_str = date + " " + time_of_day
time_delta = pd.to_datetime(time_str, format=r"%Y-%m-%d %H:%M:%S").diff()
to_winter = time_delta < pd.Timedelta(minutes=0)
to_summer = time_delta > pd.Timedelta(minutes=60)

# Define time zone string for each time stamp
time_zone = pd.Series(index=time_str.index, dtype="string", name="Timezone")
time_zone[to_winter] = "+0100"
time_zone[to_winter.shift(-1, fill_value=False)] = "+0200"
time_zone[to_summer] = "+0200"
time_zone[to_summer.shift(-1, fill_value=False)] = "+0100"
time_zone = time_zone.ffill().bfill()

# Convert time and date to timestamp respecting daylight saving
time_tz_aware_str = time_str + time_zone
time_utc = pd.to_datetime(time_tz_aware_str, format=r"%Y-%m-%d %H:%M:%S%z", utc=True)

In [ ]:
# Values are stored in column 'Zaehlerstand' of format '1234,123 ' with unit 50 kWh
raw_values_str = combined_csv_df["Zaehlerstand"].str.replace(",", ".").rename("Values")
raw_values = raw_values_str.astype(float) * unit_factor

# Use time as index
raw_values.index = time_utc

# Make timestamps time zone aware so that grouping by hours works as expected
raw_values = raw_values.tz_convert("Europe/Berlin")

# Filter values where the correct comma was dropped
# E.g. 789123 instead of 789.123
missing_comma = raw_values > (anomalie_limit * unit_factor)
values = raw_values[~missing_comma]

# Filter values that where a high bit was interpreted as low bit causes the value to be lower than the previous value
# E.g. 749.123 instead of 789.123
while not values.is_monotonic_increasing:
    values = values[~(values.diff() < 0)]

# Print stat on removed values
values_count = len(raw_values)
outlier_count = values_count - len(values)
print(f"Removed {outlier_count} ({outlier_count / values_count:.2%}) outlier values.")

In [ ]:
# Resample values to 1 minute frequency and interpolate in between
# Create new timestamp index
time = values.index
freq = "1min"
resampled_time_min = time.min().ceil(freq=freq)
resampled_time_max = time.max().floor(freq=freq)
resampled_time = pd.date_range(resampled_time_min, resampled_time_max, freq=freq)

# Create an index with old timestamps and resampled timestamps
combined_time = time.union(resampled_time)

# Interpolate values in between old timestamps
combined_values = values.reindex(combined_time).interpolate()

# Drop old timestamps
resampled_values = combined_values.reindex(resampled_time)

In [ ]:
# Get usage per minute
minutely_usage = resampled_values.diff()

# Get usage per hour, which is also the avergae per hour current
hourly_current = minutely_usage.resample("1h").sum()
grouped_hourly_current = hourly_current.groupby(
    [hourly_current.index.hour, hourly_current.index.weekday]
)

# Get current per quarter hour
quarter_hourly_current = minutely_usage.resample("15min").sum() * 4
grouped_quarter_hourly_current = quarter_hourly_current.groupby(
    [
        (quarter_hourly_current.index.hour + quarter_hourly_current.index.minute / 60),
        quarter_hourly_current.index.weekday,
    ]
)

# Get usage per day
daily_usage = minutely_usage.resample("1d").sum()
grouped_weekday_usage = daily_usage.groupby(daily_usage.index.weekday)

# Get usage per month
monthly_usage = minutely_usage.resample("1ME").sum()
grouped_month_usage = monthly_usage.groupby(monthly_usage.index.month)

In [ ]:
# German labels for calendar plots
daylabels = ["Mo", "Di", "Mi", "Do", "Fr", "Sa", "So"]
monthlabels = [
    "Jan",
    "Feb",
    "Mär",
    "Apr",
    "Mai",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Okt",
    "Nov",
    "Dez",
]

In [ ]:
title = f"Täglicher {usage_name} in {usage_unit}"
calplot.calplot(
    daily_usage.tz_localize(None),
    edgecolor=None,
    cmap="YlGn",
    linewidth=0,
    suptitle=title,
    daylabels=daylabels,
    monthlabels=monthlabels,
)

In [ ]:
# Percentiles for aggregation. 'None' is used to calculate mean value
p_values = [0.5, 0.75, 0.9, None]

for p_value in p_values:
    ax = plt.gca()
    if p_value is None:
        data = grouped_hourly_current.mean().unstack()
        aggregation = "Durchschnitt"
    else:
        data = grouped_hourly_current.quantile(p_value).unstack()
        if p_value == 0.5:
            aggregation = "Median"
        else:
            aggregation = f"p{int(p_value * 100)}"
    c = ax.pcolor(data, cmap="YlGn", vmin=0)

    for side in ("top", "right", "left", "bottom"):
        ax.spines[side].set_visible(False)
    for axis in (ax.xaxis, ax.yaxis):
        axis.set_tick_params(which="both", length=0)

    ax.set_title(f"Stündlicher {usage_name} ({aggregation}) in {usage_unit}")
    ax.set_xticks(data.columns + 0.5)
    ax.set_xticklabels(daylabels)

    ax.invert_yaxis()
    ax.set_yticks(range(0, 24, 3))
    ax.yaxis.set_major_formatter("{x} Uhr")

    plt.colorbar(c, ax=ax)
    plt.show()

In [ ]:
# Percentiles for aggregation. 'None' is used to calculate mean value
p_values = [0.5, 0.9, None]

for p_value in p_values:
    ax = plt.gca()
    if p_value is None:
        data = grouped_hourly_current.mean().unstack()
        aggregation = "Durchschnitt"
    else:
        data = grouped_hourly_current.quantile(p_value).unstack()
        if p_value == 0.5:
            aggregation = "Median"
        else:
            aggregation = f"p{int(p_value * 100)}"

    ax.set_title(f"{current_name} ({aggregation}) in {current_unit}")
    ax.plot(data, label=daylabels)

    ax.set_xticks(range(0, 24, 3))
    ax.xaxis.set_major_formatter("{x} Uhr")

    # ax.set_yticks(range(0, round(data.max(axis=None)) + 1, 5))
    ax.margins(x=0)
    ax.grid()
    ax.legend()

    plt.show()

In [ ]:
weekday_usage_mean = grouped_weekday_usage.mean()

ax = plt.gca()
ax.set_title(f"Täglicher {usage_name} (Durchschnitt) in {usage_unit}")
ax.bar(weekday_usage_mean.index, weekday_usage_mean)

ax.set_xticks(range(0, 7))
ax.set_xticklabels(daylabels)

ax.yaxis.grid(True)

In [ ]:
month_usage_mean = grouped_month_usage.mean()

ax = plt.gca()
ax.set_title(f"Monatlicher {usage_name} (Durchschnitt) in {usage_unit}")
ax.plot(month_usage_mean)
ax.set_xticks(range(1, 13))
ax.set_xticklabels(monthlabels)
ax.grid()
ax.margins(x=0)